In [1]:
import argparse
import cv2
import torch
import numpy as np
import torch.backends.cudnn as cudnn
import time
from pathlib import Path
from numpy import random
from models.experimental import attempt_load
from utils.datasets import LoadStreams, LoadImages
from utils.general import check_img_size, check_requirements, check_imshow, non_max_suppression, apply_classifier, \
    scale_coords, xyxy2xywh, strip_optimizer, set_logging, increment_path
from utils.plots import plot_one_box
from utils.torch_utils import select_device, load_classifier, time_synchronized, TracedModel

ModuleNotFoundError: No module named 'models'

In [ ]:
def letterbox(img, new_shape=(640, 640), color=(114, 114, 114), auto=True, scaleFill=False, scaleup=True, stride=32):
    shape = img.shape[:2]  
    if isinstance(new_shape, int):
        new_shape = (new_shape, new_shape)
    
    r = min(new_shape[0] / shape[0], new_shape[1] / shape[1]) 
    if not scaleup:  r = min(r, 1.0) 

    ratio = r, r 
    new_unpad = int(round(shape[1] * r)), int(round(shape[0] * r))
    dw, dh = new_shape[1] - new_unpad[0], new_shape[0] - new_unpad[1] 
    if auto:  
        dw, dh = np.mod(dw, stride), np.mod(dh, stride)  
    elif scaleFill:  
        dw, dh = 0.0, 0.0
        new_unpad = (new_shape[1], new_shape[0])
        ratio = new_shape[1] / shape[1], new_shape[0] / shape[0]  

    dw /= 2; dh /= 2

    if shape[::-1] != new_unpad:  
        img = cv2.resize(img, new_unpad, interpolation=cv2.INTER_LINEAR)
    top, bottom = int(round(dh - 0.1)), int(round(dh + 0.1))
    left, right = int(round(dw - 0.1)), int(round(dw + 0.1))
    img = cv2.copyMakeBorder(img, top, bottom, left, right, cv2.BORDER_CONSTANT, value=color)  
    return img, ratio, (dw, dh)

In [ ]:
# opt字典: 儲存參數
opt  = { "weights": "weights/yolov7-tiny.pt",  # 權重模型位置
         "yaml"   : "data/coco.yaml",     # 標籤名稱
         "img-size": 640,                 # 輸入影像大小
         "conf-thres": 0.5,               # 預測的置信度臨界值 
         "iou-thres" : 0.5,               # 非最大抑制的IoU臨界值
         "device" : 'CPU',  # 設備是否(不)啟用GPU(CPU) 0 or cpu
       }

In [ ]:
with torch.no_grad():
    weights, imgsz = opt['weights'], opt['img-size']
    device = select_device(opt['device'])
    half = device.type != 'cpu'
    model = attempt_load(weights, map_location=device)  
    stride = int(model.stride.max())  
    imgsz = check_img_size(imgsz, s=stride)  
    names = model.module.names if hasattr(model, 'module') else model.names
    colors = [[random.randint(0, 255) for _ in range(3)] for _ in names]
    model(torch.zeros(1, 3, imgsz, imgsz).to(device).type_as(next(model.parameters())))
    torch.cuda.empty_cache() 

Fusing layers... 


C:\Users\AIoT\.conda\envs\YOLOv7\lib\site-packages\torch\functional.py:568: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\TensorShape.cpp:2228.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [ ]:
def yolov7_inference(img0):
    with torch.no_grad():
        img = letterbox(img0, imgsz, stride=stride)[0]
        img = img[:, :, ::-1].transpose(2, 0, 1)  
        img = np.ascontiguousarray(img)
        img = torch.from_numpy(img).to(device).float() / 255.0
        if img.ndimension() == 3:
            img = img.unsqueeze(0)
        pred = model(img, augment= False)[0]
        pred = non_max_suppression(pred, opt['conf-thres'], opt['iou-thres'], agnostic= False)

        for i, det in enumerate(pred):
            s = '' ; s += '%gx%g ' % img.shape[2:]  
            if len(det):
                det[:, :4] = scale_coords(img.shape[2:], det[:, :4], img0.shape).round()
                for c in det[:, -1].unique():
                    n = (det[:, -1] == c).sum()  
                    s += f"{n} {names[int(c)]}{'s' * (n > 1)}, "  
                for *xyxy, conf, cls in reversed(det):
                    label = f'{names[int(cls)]} {conf:.2f}'
                    plot_one_box(xyxy, img0, label=label, color=colors[int(cls)], line_thickness=3)
    torch.cuda.empty_cache()
    return img0

In [ ]:
source_image_path = 'inference/images/horses.jpg'
img = cv2.imread(source_image_path)
img = yolov7_inference(img)
cv2.imshow('My Image', img)
cv2.waitKey(0)
cv2.destroyAllWindows()